In [ ]:
##importing Basic libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model, naive_bayes
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
import bokeh as bk
from matplotlib import pyplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error,r2_score
import plotly
import chart_studio.plotly as py
from chart_studio.plotly import plot, iplot 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
import pybaseball as pyb
from pybaseball import statcast
from pybaseball import statcast_pitcher
from pybaseball import statcast_batter
from pybaseball import statcast_pitcher_exitvelo_barrels
from pybaseball import statcast_batter_exitvelo_barrels
from pybaseball import statcast_batter_expected_stats
from pybaseball import statcast_pitcher_expected_stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import pwlf as pwlf
#import matplotlib.backends
import matplotlib as mpl

In [ ]:
df3 = pd.read_csv("/Users/johndavis/Desktop/College_Updated_5_14.csv",low_memory=False)

In [ ]:
#find the highest value in RelSpeed column
df3['RelSpeed'].max()

In [ ]:
#Create a new dataframe with only the columns we want for Analysis
dft = df3[['Pitcher','PitcherTeam','TaggedPitchType','PitchCall','PlayResult','RelSpeed','SpinRate','RelHeight','RelSide','Extension','InducedVertBreak', 'HorzBreak','PlateLocHeight','PlateLocSide']]

In [ ]:
#fastballs = 'Fastball', 'Sinker', 'TwoSeamFastBall', 'FourSeamFastBall', 'OneSeamFastBall'
dfb2 = dft[dft.TaggedPitchType.isin(['Fastball', 'Sinker', 'TwoSeamFastBall', 'FourSeamFastBall', 'OneSeamFastBall'])]
#sliders = 'Slider', 'Cutter'
dsl2 = dft[dft.TaggedPitchType.isin(['Slider', 'Cutter'])]
#curveballs = 'Curveball', 'KnuckleCurve'
dcb2 = dft[dft.TaggedPitchType.isin(['Curveball', 'KnuckleCurve'])]
#changeups = 'Changeup', 'Splitter', 'Forkball', 'Screwball'
dch2 = dft[dft.TaggedPitchType.isin(['ChangeUp'])]



In [ ]:
#create a new column called ABS_Horizontal with the absolute value of HorzBreak  in dfb2
dfb2['ABS_Horizontal'] = abs(dfb2['HorzBreak'])
dsl2['ABS_Horizontal'] = abs(dsl2['HorzBreak'])
dcb2['ABS_Horizontal'] = abs(dcb2['HorzBreak'])
dch2['ABS_Horizontal'] = abs(dch2['HorzBreak'])
#create a new column called ABS_relside with the absolute value of RelSide in dfb2
dfb2['ABS_RelSide'] = abs(dfb2['RelSide'])
dsl2['ABS_RelSide'] = abs(dsl2['RelSide'])
dcb2['ABS_RelSide'] = abs(dcb2['RelSide'])
dch2['ABS_RelSide'] = abs(dch2['RelSide'])
#create a new column called differential break which is the absolute value of InducedVertBreak - ABS_Horizontal
dfb2['differential_break'] = abs(dfb2['InducedVertBreak'] - dfb2['ABS_Horizontal'])
dsl2['differential_break'] = abs(dsl2['InducedVertBreak'] - dsl2['ABS_Horizontal'])
dcb2['differential_break'] = abs(dcb2['InducedVertBreak'] - dcb2['ABS_Horizontal'])
dch2['differential_break'] = abs(dch2['InducedVertBreak'] - dch2['ABS_Horizontal'])


In [ ]:
#delete all rows with nan values
dfb2 = dfb2.dropna()
dsl2 = dsl2.dropna()
dcb2 = dcb2.dropna()
dch2 = dch2.dropna()


In [ ]:
#reset index    
dfb2 = dfb2.reset_index(drop=True)
dsl2 = dsl2.reset_index(drop=True)
dcb2 = dcb2.reset_index(drop=True)
dch2 = dch2.reset_index(drop=True)

In [ ]:
#PitchCall unique values
dft['PitchCall'].unique()

In [ ]:
#change 'SwinginStrike', 'StriekSwinging' in PitchCall to 'StrikeSwinging'
dft['PitchCall'] = dft['PitchCall'].replace(['SwinginStrike', 'StriekSwinging'], 'StrikeSwinging')

In [ ]:
#new dataframe with only PitchCall StrikeSwinging, InPlay, FoulBall and call it df_Swing
df_Swing = dft[(dft['PitchCall'] == 'StrikeSwinging') | (dft['PitchCall'] == 'InPlay') | (dft['PitchCall'] == 'FoulBall')]

In [ ]:
#create a dummy variable for PitchCall with StrikeSwinging = 1 and InPlay, FoulBall = 0
df_Swing['Whiff'] = df_Swing['PitchCall'].replace(['StrikeSwinging'], 1)
df_Swing['Whiff'] = df_Swing['PitchCall'].replace(['InPlay', 'FoulBall'], 0)

In [ ]:
#replace strike swinging with 1 and inplay, foulball with 0
df_Swing['Whiff'] = df_Swing['Whiff'].replace(['StrikeSwinging'], 1)

In [ ]:
#create a new column called ABS_Horizontal with the absolute value of HorzBreak
df_Swing['ABS_Horizontal'] = abs(df_Swing['HorzBreak'])
#create a new column called ABS_RelSide with the absolute value of RelSide
df_Swing['ABS_RelSide'] = abs(df_Swing['RelSide'])

In [ ]:
#create a new column called differential break which is the absolute value of InducedVertBreak - ABS_Horizontal
df_Swing['differential_break'] = abs(df_Swing['InducedVertBreak'] - df_Swing['ABS_Horizontal'])

In [ ]:
#taggedpitchtype unique values
df_Swing['TaggedPitchType'].unique()

In [ ]:
#remove any row with Nan for any of the columns
df_Swing = df_Swing.dropna()
#reset the index
df_Swing = df_Swing.reset_index(drop=True)

In [ ]:
#seperate dataframes for each pitch type 
#fastballs = 'Fastball', 'Sinker', 'TwoSeamFastBall', 'FourSeamFastBall', 'OneSeamFastBall'
df_Fastball = df_Swing[(df_Swing['TaggedPitchType'] == 'Fastball') | (df_Swing['TaggedPitchType'] == 'Sinker') | (df_Swing['TaggedPitchType'] == 'TwoSeamFastBall') | (df_Swing['TaggedPitchType'] == 'FourSeamFastBall') | (df_Swing['TaggedPitchType'] == 'OneSeamFastBall')]
df_Fastball = df_Fastball.reset_index(drop=True)

In [ ]:
#curveballs = 'Curveball', 'KnuckleCurve'
df_Curveball = df_Swing[(df_Swing['TaggedPitchType'] == 'Curveball') | (df_Swing['TaggedPitchType'] == 'KnuckleCurve')]
df_Curveball = df_Curveball.reset_index(drop=True)

In [ ]:
#df_Slider = 'Slider'
df_Slider = df_Swing[(df_Swing['TaggedPitchType'] == 'Slider')]
df_Slider = df_Slider.reset_index(drop=True)

In [ ]:
#df_Changeup = 'Changeup'
df_Changeup = df_Swing[(df_Swing['TaggedPitchType'] == 'ChangeUp')]
df_Changeup = df_Changeup.reset_index(drop=True)

In [ ]:
df_Swing.columns

In [ ]:
#create the FB expected whiff rate
X = df_Fastball[['RelSpeed', 'SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']]
y = df_Fastball['Whiff']

In [ ]:
#create the CB expected whiff rate
X1 = df_Curveball[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y1 = df_Curveball['Whiff']

In [ ]:
#create the SL expected whiff rate
X2 = df_Slider[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y2 = df_Slider['Whiff']

In [ ]:
#create the CH expected whiff rate
X3 = df_Changeup[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y3 = df_Changeup['Whiff']

In [ ]:
#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.25, random_state=101)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25, random_state=101)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.25, random_state=101)



In [ ]:
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create the neural network model
    model = Sequential()
    model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

    return model

In [ ]:
#xgboost for fastball
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

In [ ]:
#random forest for fastball
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

In [ ]:
fastball_model = train_model(X, y)

In [ ]:
#xgboost for curveball
xgb_model1 = XGBClassifier()
xgb_model1.fit(X1_train, y1_train)

In [ ]:
#random forest for curveball
rfc1 = RandomForestClassifier(n_estimators=100)
rfc1.fit(X1_train, y1_train)

In [ ]:
#CURVEBALL MODEL
curveball_model = train_model(X1, y1)

In [ ]:
#xgboost for slider
xgb_model2 = XGBClassifier()
xgb_model2.fit(X2_train, y2_train)

In [ ]:
#random forest for slider
rfc2 = RandomForestClassifier(n_estimators=100)
rfc2.fit(X2_train, y2_train)


In [ ]:
#SLIDER MODEL
slider_model = train_model(X2, y2)

In [ ]:
#xgboost for changeup
xgb_model3 = XGBClassifier()
xgb_model3.fit(X3_train, y3_train)

In [ ]:
#random forest for changeup
rfc3 = RandomForestClassifier(n_estimators=100)
rfc3.fit(X3_train, y3_train)

In [ ]:
#CHANGEUP MODEL
changeup_model = train_model(X3, y3)

In [ ]:
#find rmse for fastball
predictions = xgb_model.predict(X_test)
print('XGBoost Fastball RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))

In [ ]:
#find rmse for fastball
predictions = rfc.predict(X_test)
print('Random Forest Fastball RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))

In [ ]:
#find rmse for Fastball nn
predictions = fastball_model.predict(X_test)
print('Neural Network Fastball RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))

In [ ]:
#create a new column in the dfb2 dataframe called xWhiff_xg and set it equal to the predicted probability of a whiff as predicted by the xgboost model
dfb2['xWhiff_xg'] = xgb_model.predict_proba(dfb2[['RelSpeed', 'SpinRate', 'differential_break', 'RelHeight', 'ABS_RelSide', 'Extension']])[:,1]


In [ ]:
#create a new column in the dfb2 dataframe called xWhiff_xg and set it equal to the predicted probability of a whiff as predicted by the xgboost model
dcb2['xWhiff_xg'] = xgb_model1.predict_proba(dcb2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])[:,1]

In [ ]:
#create a new column in the dsl2 dataframe called xWhiff and use the model tp predict the expected whiff rate given the same variables
dsl2['xWhiff_xg'] = xgb_model2.predict_proba(dsl2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])[:,1]

In [ ]:
#create a new column in the dch2 dataframe called xWhiff and use the model tp predict the expected whiff rate given the same variables
dch2['xWhiff_xg'] = xgb_model3.predict_proba(dch2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])[:,1]

In [ ]:
#create a new column in the dfb2 dataframe called xWhiff_rf and set it equal to the predicted probability of a whiff as predicted by the random forest model
dfb2['xWhiff_rf'] = rfc.predict_proba(dfb2[['RelSpeed', 'SpinRate', 'differential_break', 'RelHeight', 'ABS_RelSide', 'Extension']])[:,1]


In [ ]:
#create a new column in the dfb2 dataframe called xWhiff_rf and set it equal to the predicted probability of a whiff as predicted by the random forest model
dcb2['xWhiff_rf'] = rfc1.predict_proba(dcb2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])[:,1]

In [ ]:
#create a new column in the dsl2 dataframe called xWhiff and use the model tp predict the expected whiff rate given the same variables
dsl2['xWhiff_rf'] = rfc2.predict_proba(dsl2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])[:,1]

In [ ]:
#create a new column in the dch2 dataframe called xWhiff and use the model tp predict the expected whiff rate given the same variables
dch2['xWhiff_rf'] = rfc3.predict_proba(dch2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])[:,1]

In [ ]:
#USE NEURAL NETWORK TO PREDICT WHIFF RATE FOR FASTBALL AND ADD TO DATAFRAME
dfb2['xWhiff_nn'] = fastball_model.predict(dfb2[['RelSpeed', 'SpinRate', 'differential_break', 'RelHeight', 'ABS_RelSide', 'Extension']])

In [ ]:
#USE NEURAL NETWORK TO PREDICT WHIFF RATE FOR CURVEBALL AND ADD TO DATAFRAME
dcb2['xWhiff_nn'] = curveball_model.predict(dcb2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])

In [ ]:
#USE NEURAL NETWORK TO PREDICT WHIFF RATE FOR SLIDER AND ADD TO DATAFRAME
dsl2['xWhiff_nn'] = slider_model.predict(dsl2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])

In [ ]:
#USE NEURAL NETWORK TO PREDICT WHIFF RATE FOR CHANGEUP AND ADD TO DATAFRAME
dch2['xWhiff_nn'] = changeup_model.predict(dch2[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])

In [ ]:
#take the average of the Three models to get the final expected whiff rate
dfb2['xWhiff'] = (dfb2['xWhiff_xg'] + dfb2['xWhiff_rf'] + dfb2['xWhiff_nn'])/3
dcb2['xWhiff'] = (dcb2['xWhiff_xg'] + dcb2['xWhiff_rf'] + dcb2['xWhiff_nn'])/3
dsl2['xWhiff'] = (dsl2['xWhiff_xg'] + dsl2['xWhiff_rf'] + dsl2['xWhiff_nn'])/3
dch2['xWhiff'] = (dch2['xWhiff_xg'] + dch2['xWhiff_rf'] + dch2['xWhiff_nn'])/3

In [ ]:
#find the mean of the xWhiff column
dfb2['xWhiff'].mean()

In [ ]:
#find the mean of the xWhiff column
dcb2['xWhiff'].mean()

In [ ]:
#find the mean of the xWhiff column
dsl2['xWhiff'].mean()

In [ ]:
#find the mean of the xWhiff column
dch2['xWhiff'].mean()

In [ ]:
#create a new column in dfb2 called stuff+ which is (xWhiff / mean of xWhiff for dfb2) * 100
dfb2['stuff+'] = (dfb2['xWhiff'] / dfb2['xWhiff'].mean()) * 100
#create a new column in dcb2 called stuff+ which is (xWhiff / mean of xWhiff for dcb2) * 100
dcb2['stuff+'] = (dcb2['xWhiff'] / dcb2['xWhiff'].mean()) * 100
#create a new column in dsl2 called stuff+ which is (xWhiff / mean of xWhiff for dsl2) * 100
dsl2['stuff+'] = (dsl2['xWhiff'] / dsl2['xWhiff'].mean()) * 100
#create a new column in dch2 called stuff+ which is (xWhiff / mean of xWhiff for dch2) * 100
dch2['stuff+'] = (dch2['xWhiff'] / dch2['xWhiff'].mean()) * 100

In [ ]:
#find correlation between each xWhiff and stuff+ column
dfb2['xWhiff_nn'].corr(dfb2['stuff+'])

In [ ]:
#calulcate the mean of stuff+ for dfb2
dfb2['stuff+'].mean()

In [ ]:
#calulcate the mean of stuff+ for dcb2
dcb2['stuff+'].mean()

In [ ]:
#calulcate the mean of stuff+ for dsl2
dsl2['stuff+'].mean()


In [ ]:
#calulcate the mean of stuff+ for dch2
dch2['stuff+'].mean()

In [ ]:
#creat a visualization of the mean of stuff+ for each pitch type 
plt.bar(['Fastball', 'Curveball', 'Slider', 'Changeup'], [dfb2['stuff+'].mean(), dcb2['stuff+'].mean(), dsl2['stuff+'].mean(), dch2['stuff+'].mean()])


In [ ]:
#reccombine the dataframes
College_Final_Stuff = pd.concat([dfb2, dsl2, dcb2, dch2], ignore_index=True)

In [ ]:
College_Final_Stuff

In [ ]:
 #group by PitcherTeam and find the mean of stuff+ for each team
df_College_Team_AVG3 = College_Final_Stuff.groupby('PitcherTeam')['stuff+'].mean()

In [ ]:
#sort the values high to low
df_College_Team_AVG3 = df_College_Team_AVG3.sort_values(ascending=False)

In [ ]:
#change PitcherTeam to team_abbr
df_College_Team_AVG3 = df_College_Team_AVG3.rename_axis('team_abbr').reset_index()

In [ ]:

df_College_Names3 = pd.read_csv("/Users/johndavis/Desktop/Trackman_Names.csv", low_memory=False)

In [ ]:
df_College_Names3

In [ ]:
#merge the two dataframes by team_abbr
df_College_Team_AVG2 = pd.merge(df_College_Team_AVG3, df_College_Names3, on='team_abbr', how='left')

In [ ]:
#only include conference, team, stuff+ and and  team_abbr,  sort by stuff+ high to low
df_College_Team_AVG2 = df_College_Team_AVG2[['conference', 'team', 'stuff+', 'team_abbr']].sort_values(by='stuff+', ascending=False)

In [ ]:
#Locate the team_abbr for the team you want to find the stuff+ for
df_College_Team_AVG2.loc[df_College_Team_AVG2['team_abbr'] == 'PAC_TIG']

In [ ]:
top_50_college_teams = df_College_Team_AVG2.head(50)

In [ ]:
top_50_college_teams

In [ ]:
#sort values low to high
bottom_50_college_teams = df_College_Team_AVG2.sort_values(by='stuff+', ascending=True)

In [ ]:
#group by conference and find the mean of stuff+ for each conference
df_College_Conference_AVG3 = df_College_Team_AVG2.groupby('conference')['stuff+'].mean()

In [ ]:
df_College_Conference_AVG3 = df_College_Conference_AVG3.sort_values(ascending=False)

In [ ]:
top_10_college_teams = df_College_Team_AVG2.head(10)

In [ ]:
top_10_conferences = df_College_Conference_AVG3.head(10)

In [ ]:
#csv to desktop
top_10_college_teams.to_csv('/Users/johndavis/Desktop/top_10_college_teams.csv')

In [ ]:
#csv to desktop
top_10_conferences.to_csv('/Users/johndavis/Desktop/top_10_conferences.csv')

In [ ]:
#csv to desktop 
College_Final_Stuff.to_csv(r'/Users/johndavis/Desktop/College_Final_Stuff_Full.csv', index = False)

In [ ]:
#group by pitcher and pitch type and take the average of stuff+, do not make it a multi index
College_Final_Stuff_avg = College_Final_Stuff.groupby(['Pitcher', 'TaggedPitchType'], as_index=False)['PitcherTeam','stuff+','RelSpeed','InducedVertBreak','HorzBreak'].mean()
#create a pitch count column which is the count of each pitch type of each pitcher from df_Swing_2
College_Final_Stuff_avg['PitchCount'] = College_Final_Stuff.groupby(['Pitcher', 'TaggedPitchType'])['TaggedPitchType'].count().values

In [ ]:
#sort College_Final_Stuff_avg by stuff+ high to low
College_Final_Stuff_avg = College_Final_Stuff_avg.sort_values(by='stuff+', ascending=False)

In [ ]:
#min pitch count of 200 for Fastball, Two-Seam, Four-Seam, Sinker and Min pitch count of 100 for Slider, Curveball and Changeup
College_Final_Stuff_avg_pitch_count = College_Final_Stuff_avg[(College_Final_Stuff_avg['TaggedPitchType'] == 'Fastball') & (College_Final_Stuff_avg['PitchCount'] >= 200) | (College_Final_Stuff_avg['TaggedPitchType'] == 'Two-Seam') & (College_Final_Stuff_avg['PitchCount'] >= 200) | (College_Final_Stuff_avg['TaggedPitchType'] == 'Four-Seam') & (College_Final_Stuff_avg['PitchCount'] >= 200) | (College_Final_Stuff_avg['TaggedPitchType'] == 'Sinker') & (College_Final_Stuff_avg['PitchCount'] >= 200) | (College_Final_Stuff_avg['TaggedPitchType'] == 'Slider') & (College_Final_Stuff_avg['PitchCount'] >= 100) | (College_Final_Stuff_avg['TaggedPitchType'] == 'Curveball') & (College_Final_Stuff_avg['PitchCount'] >= 100) | (College_Final_Stuff_avg['TaggedPitchType'] == 'Changeup') & (College_Final_Stuff_avg['PitchCount'] >= 100)]

In [ ]:
College_Final_Stuff_avg_pitch_count.head(25)

In [ ]:
#locate Skenes, Paul
College_Final_Stuff_avg.loc[College_Final_Stuff_avg['Pitcher'] == 'Skenes, Paul']

In [ ]:
#College_Final_Stuff_avg to desktop csv
College_Final_Stuff_avg.to_csv(r'/Users/johndavis/Desktop/College_Final_Stuff_Avg.csv', index = False)

In [ ]:
#new dataframe with minimum 100 pitches thrown
College_Final_Stuff_avg_100_pitch_min = College_Final_Stuff_avg[College_Final_Stuff_avg['PitchCount'] >= 100]

In [ ]:
#sort by stuff+ descending
College_Final_Stuff_avg_100_pitch_min = College_Final_Stuff_avg_100_pitch_min.sort_values(by=['stuff+'], ascending=False)

In [ ]:
#create a new dataframe out of College_Final_Stuff_avg that averages the stuff+ for each pitcher and combines the pitch types based on the average stuff+ for each pitch and weighted by the pitch count
College_Final_Stuff_avg_AVG2 = College_Final_Stuff_avg.groupby(['Pitcher'], as_index=False)['stuff+'].mean()
#add the pitch count columns to the dataframe from each pitch add them together
College_Final_Stuff_avg_AVG2['PitchCount'] = College_Final_Stuff_avg.groupby(['Pitcher'])['PitchCount'].sum().values



In [ ]:
#sort by stuff+ descending
College_Final_Stuff_avg_AVG2 = College_Final_Stuff_avg_AVG2.sort_values(by=['stuff+'], ascending=False)

In [ ]:
#make a new dataframe with 400 pitch minimum for College_Final_Stuff_avg_AVG2
College_Final_Stuff_avg_AVG2_400_pitch_min = College_Final_Stuff_avg_AVG2[College_Final_Stuff_avg_AVG2['PitchCount'] >= 400]

In [ ]:
#sort by stuff+ descending
College_Final_Stuff_avg_AVG2_400_pitch_min = College_Final_Stuff_avg_AVG2_400_pitch_min.sort_values(by=['stuff+'], ascending=False)

In [ ]:
College_Final_Stuff_avg_AVG2_400_pitch_min.head(25)